# Week 7

In [1]:
%pip install surprise pandas scikit-learn matplotlib seaborn
%pip install fastparquet pyarrow
%pip install numpy==1.26.4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [69]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np  


In [ ]:
test_df = pd.read_csv('csv_files/preprocessed_test_video_games.csv')
train_df = pd.read_csv('csv_files/preprocessed_train_video_games.csv')
items_highly_rated_freq = pd.read_csv('csv_files/item_highly_rated_freq.csv')

Based on the frequency of the most rated items computed in Week 6,
implement the TopPop Recommender System, which always recommends
the same top-k items sorted decreasingly by the number of “high” ratings
(e.g., → 3) in the training split (train video games.parquet).

In [72]:
# implement based line TopPop recommender system
top_pop_items = items_highly_rated_freq.head(10)
# just recommend the top 10 most popular highly rated (defined as having a rating >= 3) items
print(top_pop_items)


      item_id  highly_rated_count  average_rating_by_item
0  B0086VPUHI                 154                4.425150
1  B00BN5T30E                 150                4.428571
2  B07YBXFDYN                 146                4.329268
3  B00BGA9WK2                 136                4.432432
4  B007CM0K86                 123                4.554688
5  B00KIWEMIG                 109                4.237705
6  B07YBWT3PK                 104                4.107438
7  B07YBXFF99                 103                4.345455
8  B014R4KYMS                 102                4.532110
9  B004HD55VK                 102                4.615385


Choose at least one neighborhood-based model and one latent factor model (both type of colaborative filtering)
that uses the observed user-item ratings in the training set to predict the
unobserved ratings. Report your choice of models.   

Does unobserved mean empty cell? There is no missing data as seen in from the data exploration

Use 5-fold cross-validation on the training set to tune the hyperparameters
of the chosen models (similarity measure and number of neighbors for the
neighborhood-based model; number of latent factors and number of epochs
for the latent factor model).

Choose an evaluation measure that is suitable for this task and justify your
motivation in using it. Report the optimal hyperparameters together with
the scores of your chosen measure, averaged over the 5 folds.
• Run the models with the optimal hyperparameters to the whole training
set

In [11]:
# convert train and test to csv files
# train_df.to_csv('train_video_games.csv', index=False)
# test_df.to_csv('test_video_games.csv', index=False) 

### Item Based KNN Colaborative Filtering




In [73]:
# create item based KNN colaborative filtering recommender system using the surprise library
# use 5-fold cross validation 
# use Root Mean Square Error (RMSE) as the evaluation metric
from surprise import Dataset, Reader
from surprise import KNNBasic, KNNBaseline
from surprise.model_selection import cross_validate, GridSearchCV

# load the data into surprise format
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(train_df[['user_id', 'item_id', 'rating']], reader)



In [74]:
# train/evaluate the algorithm using grind search 5 fold cross validation
params = {'num_neighbors':[5, 10, 20, 25], 'epochs':[5, 10, 15, 20], 'sim_options': {'name': ['cosine', 'pearson'], 'user_based': [False]}}

cv_results = GridSearchCV(KNNBasic, params, cv=5)

cv_results.fit(data) # what does .fit do? .fit trains the model on the data. In this case, 
# it will train the KNNBasic algorithm with the specified parameters using 5-fold cross validation. 
# for each combination of parameters, it does a 5-fold cross validation and aggegates the results rmse and mae
# It will evaluate the performance of each combination of parameters and store the results in cv_results.


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computi

In [76]:
print(cv_results.best_params)
# what is the mae for the best parameters?
print(cv_results.best_score)

{'rmse': {'num_neighbors': 5, 'epochs': 5, 'sim_options': {'name': 'cosine', 'user_based': False}}, 'mae': {'num_neighbors': 5, 'epochs': 5, 'sim_options': {'name': 'cosine', 'user_based': False}}}
{'rmse': 1.0236965914880671, 'mae': 0.6956810377218265}


This is LABELED training data. So we can calculate RMSE on the ground truth.

The two similarity measure for nearest neighbor is Cosine Similarity, Pearson coefficent. Cosine Similarity measure is used for item based. Why?     
We can also used euclidean distance or dot product if we wanted to.

In [77]:
# run the optimal hyperparameters to the whole training set
best_knn = KNNBasic(num_neighbors=5, epochs=5, sim_options={'name': 'cosine', 'user_based': False})
best_knn.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [80]:
# save model using pickle
import pickle
with open('best_knn_model.pkl', 'wb') as f:
    pickle.dump(best_knn, f)

In [81]:
# provide one example of a prediction for user_id one user and one item in the dataset
uid = str(train_df['user_id'].iloc[0])  # raw user id (as in the train_df)
iid = str(train_df['item_id'].iloc[0])  # raw item id (as in the train_df)
pred = best_knn.predict(uid, iid)
print(pred)

user: AHOGCWGRSFQ6YZH6QLYUMNQ4N3KA item: B00ZM5OXD8 r_ui = None   est = 3.86   {'actual_k': 33, 'was_impossible': False}


In [86]:
# use the final model to  rank the non-rated items for each user based on the predictions of best_knn

recommendations = {}
for user_id in train_df['user_id'].unique():
    # get the items that the user has rated
    rated_items = train_df[train_df['user_id'] == user_id]['item_id'].tolist()
    
    # get the items that the user has not rated
    unrated_items = train_df[~train_df['item_id'].isin(rated_items)]['item_id'].unique().tolist()
    
    # predict the ratings for the unrated items
    predictions = []
    for item_id in unrated_items:
        pred = best_knn.predict(user_id, item_id)
        predictions.append((item_id, pred.est))
    
    # sort the predictions by rating and get the top 10 recommendations
    predictions.sort(key=lambda x: x[1], reverse=True)
    recommendations[user_id] = [(item_id, round(rating, 3)) for item_id, rating in predictions[:10]]


In [88]:
# print the recommendations for the first 5 users
for user_id in list(recommendations.keys())[:5]:
    print(f"Recommendations for user {user_id}: {recommendations[user_id]}")

Recommendations for user AHOGCWGRSFQ6YZH6QLYUMNQ4N3KA: [('B001TOQ8UW', 5), ('B00005Q8LR', 5), ('B001EYUNE4', 5), ('B0009VXAM0', 5), ('B00GM5TPSA', 5), ('B001ELJEJM', 5), ('B00001X50M', 5.0), ('B001G7PS4Y', 4.911), ('B00DB2BI00', 4.91), ('B00104KCF8', 4.851)]
Recommendations for user AEKOQRDUY64SGH4PONBFTSIM2I2Q: [('B007VTVRFA', 5), ('B00DB2BI00', 5), ('B017V6YVDC', 4.799), ('B015KBXBM2', 4.798), ('B00005Q8LR', 4.752), ('B001EYUXIK', 4.75), ('B001EYUS7G', 4.749), ('B00005O0I2', 4.728), ('B00G6GQL0A', 4.715), ('B001EYUW3Q', 4.714)]
Recommendations for user AFA43JCV3C72LHM5BIVPV7UEJ2CA: [('B08JHZHWZ3', 5), ('B08MBKQWHB', 5), ('B08F4C6HCD', 5), ('B00J4Y6L2Y', 5), ('B004RMK57U', 5), ('B00MB1I3FU', 5), ('B06XHVJY3T', 5), ('B00MOR1A7Y', 5), ('B00CMQTUY2', 5), ('B08F97Q8HS', 5)]
Recommendations for user AFD3QCEDODUYBYMQGA6NWILJW7KA: [('B00O3JSRHW', 5), ('B0031SWWPO', 5), ('B09V5R5LSZ', 5), ('B0013RATNM', 5), ('B000ZK7ZOE', 5), ('B07NKN4VR4', 5), ('B0051D8PGM', 5), ('B00DC7G0GG', 5), ('B00B7UNO

### SVD 
SVD is a matrix factorization technique that decomposes the user-item interaction matrix into latent factors.


MAE is mean absolute error performance metric for evaluating the accuracy of a recommender system. It measures the average absolute difference between the predicted ratings and the actual ratings. A lower MAE indicates better performance, as it means that the predictions are closer to the true ratings. MAE is commonly used in recommender systems to assess how well the model is predicting user preferences.


We also want to minimize Root Mean Squared Error (if its an error metric, minimize it)

which performance metric is better for evaluating recommender systems? RMSE or MAE?
RMSE is more sensitive to outliers than MAE, so it may be more appropriate for evaluating recommender systems that have a lot of outliers in the ratings. MAE is less sensitive to outliers, 
So it may be more appropriate for evaluating recommender systems that have fewer outliers in the ratings. Ultimately, the choice of performance metric depends on the specific characteristics of the dataset and the goals of the evaluation.

In [89]:
# hyperparameters for SVD can be tuned using grid search
# reg_all means regularization term for all parameters, n_factors is the number of latent factors
# what does latent factors mean? Latent factors are the underlying features that explain the observed ratings.
from surprise import SVD
params_svd = {'n_factors': [25, 50, 100, 150, 175], 'n_epochs': [20, 30, 40, 50], 'lr_all': [0.002, 0.005, 0.01], 'reg_all': [0.02, 0.05, 0.1]}
cv_results_svd = GridSearchCV(SVD, params_svd, cv=5)


I will use MAE as performance metrics. Root mean squared error gives more weight to outliers but we don't have a lot of outlier ratings.

cv does what exactly?  
cv creates 5 folds of the data, trains the model on 4 folds and tests on the last fold  
each fold is run with the same hyperparameters, but different test fold   
what does the cv tell us? it tells us the average performance of the   model with the given hyperparameters across the 5 folds,   
which gives us an estimate of how well the model will perform on unseen data.  
make sure folds are random and stratified  

In [90]:
cv_results_svd.fit(data)


In [91]:
# print the average MAE
print(cv_results_svd.best_params['mae'])
print(cv_results_svd.best_score['mae'])

{'n_factors': 25, 'n_epochs': 50, 'lr_all': 0.005, 'reg_all': 0.05}
0.6776888140051007


In [93]:
# Run the models with the optimal hyperparameters to the whole training set.
best_svd = SVD(n_factors=25, n_epochs=50, lr_all=0.005, reg_all=0.05)
best_svd.fit(data.build_full_trainset())

In [94]:
# save the model using pickle
with open('best_svd_model.pkl', 'wb') as f:
    pickle.dump(best_svd, f)

In [95]:
# Use the final model to rank the non-rated items for each user
recommendations_svd = {}
for user_id in train_df['user_id'].unique():
    # get the items that the user has rated
    rated_items = train_df[train_df['user_id'] == user_id]['item_id'].tolist()
    
    # get the items that the user has not rated
    unrated_items = train_df[~train_df['item_id'].isin(rated_items)]['item_id'].unique().tolist()
    
    # predict the ratings for the unrated items
    predictions = []
    for item_id in unrated_items:
        pred = best_svd.predict(user_id, item_id)
        predictions.append((item_id, pred.est))
    
    # sort the predictions by rating and get the top 10 recommendations
    predictions.sort(key=lambda x: x[1], reverse=True)
    recommendations_svd[user_id] = [(item_id, round(rating,3)) for item_id, rating in predictions[:10]]

In [97]:
# print the recommendations for the first 5 users
for user_id in list(recommendations_svd.keys())[:5]:
    print(f"Recommendations for user {user_id}: {recommendations_svd[user_id]}")


Recommendations for user AHOGCWGRSFQ6YZH6QLYUMNQ4N3KA: [('B07YBXFDYN', 5), ('B00G2EVF3E', 5), ('B001EYUQT6', 5), ('B00KR2BK1E', 4.965), ('B09KTV12WR', 4.963), ('B01M1DN3OO', 4.911), ('B09FBW2988', 4.859), ('B004HILZUU', 4.856), ('B00DC7G0GG', 4.852), ('B01N3ASPNV', 4.851)]
Recommendations for user AEKOQRDUY64SGH4PONBFTSIM2I2Q: [('B0191J0RXA', 5), ('B07895QZBF', 4.944), ('B00000JRSB', 4.931), ('B003ZSN600', 4.911), ('B019EWMXWC', 4.898), ('B0002A6CQ4', 4.83), ('B001C6GVI6', 4.796), ('B07BLRF329', 4.789), ('B001ELJE6K', 4.789), ('B00OBZNI0O', 4.781)]
Recommendations for user AFA43JCV3C72LHM5BIVPV7UEJ2CA: [('B004HD55VK', 5), ('B00DC7G0GG', 5), ('B009GE437W', 5), ('B00MOR0FLG', 5), ('B07X56RNBY', 5), ('B004RMK5QG', 5), ('B0050SXVK8', 5), ('B00DBDPOZ4', 5), ('B007X5103Q', 5), ('B009GE3WUQ', 5)]
Recommendations for user AFD3QCEDODUYBYMQGA6NWILJW7KA: [('B000ZK7ZOE', 5), ('B004HD55VK', 5), ('B07NKN4VR4', 5), ('B00DC7G0GG', 5), ('B009GE437W', 5), ('B00MOR0FLG', 5), ('B006VR689I', 5), ('B07X56RN

# Week 8 